In [1]:
import pandas as pd
import numpy as np

# 使用相对路径加载CSV文件
csv_file_path = "main.csv"

# 使用pandas加载CSV文件到DataFrame
data = pd.read_csv(csv_file_path)

# 打印DataFrame以查看数据
data

,from,to
0,1,2
1,1,3
2,2,4
3,3,5
4,1,3
5,2,3
6,2,1
7,2,2
8,2,3
9,1,3


In [2]:
lag_period = 0
data['to_lagged'] = data['to'].shift(lag_period)
data

,from,to,to_lagged
0,1,2,2
1,1,3,3
2,2,4,4
3,3,5,5
4,1,3,3
5,2,3,3
6,2,1,1
7,2,2,2
8,2,3,3
9,1,3,3


In [3]:
transition_table = pd.pivot_table(data, index='from', columns='to', aggfunc='size', fill_value=0)

In [4]:
transition_table

to,1,2,3,4,5
from,,,,,
1,0,1,3,0,0
2,1,1,3,1,0
3,0,1,0,0,1
4,2,1,0,0,0
5,1,1,0,0,1


In [5]:
# total_events = len(data)
# transition_table = pd.pivot_table(data, index='from', columns='to', aggfunc='size', fill_value=0)
# transition_table = transition_table / total_events

# transition_table

In [6]:
# 计算事件的总数
total_events = len(data)

# 创建一个空的Z-score表格
z_score_table = pd.DataFrame(
    index=transition_table.index, columns=transition_table.columns
)
# 计算Z-score并填充表格
for from_value in transition_table.index:
    for to_value in transition_table.columns:
        # 事件 g 到事件 t 的次數
        f_g_t = transition_table.at[from_value, to_value]
        # print(f_g_t)
        # 事件 g 的總數
        f_g = transition_table.loc[from_value].sum()
        # 發生事件 g 的機率
        p_g = f_g / total_events
        # 發生事件 t 的機率
        p_t = transition_table[to_value].sum() / total_events
        # 计算Z-score
        z_score = (f_g_t - f_g * p_t) / np.sqrt(f_g * p_t * (1 - p_t) * (1 - p_g))
        # 将Z-score填充到表格
        z_score_table.at[from_value, to_value] = z_score

# 显示Z-score表格
z_score_table

to,1,2,3,4,5
from,,,,,
1,-1.212183,-0.140642,2.004459,-0.550019,-0.801784
2,-0.400892,-0.744208,1.06066,1.455214,-1.06066
3,-0.801784,0.744208,-1.06066,-0.363803,1.856155
4,2.02837,0.235339,-1.341641,-0.460179,-0.67082
5,0.507093,0.235339,-1.341641,-0.460179,1.341641
